In [1]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,24 mins 05 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,"7 days, 17 hours and 12 minutes"
H2O cluster name:,H2O_from_python_msche_z22ubu
H2O cluster total nodes:,1
H2O cluster free memory:,1.205 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [105]:
data = h2o.import_file("C:\Daten\ML\Person\data.csv")

splits = data.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [107]:
#data.head()
#y = ['P7','P8','P9']
x = data.columns
x.remove('source')
x.remove('country')
#x.remove('P7')
#x.remove('P8')
x.remove('P9')
#print(x)

In [108]:
aml = H2OAutoML(max_runtime_secs = 360)

In [109]:
aml.train(y = 'P9', x = x, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [110]:
pred = aml.predict(train)
#print(pred.head())
#print(test.head())

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [111]:
#pred.describe()
#perf = aml.leader.model_performance(test)
#perf

In [119]:
import math

df = train.as_data_frame(0)
pre = pred.as_data_frame(0)

count = 0
value = 0
valuePre = 0
dif = 0

for x in df:
    if count > 0:
        valuePre += float(pre[count][0])
        value += int(x[2])
        dif += abs(abs(float(x[2]))-abs(float(pre[count][0])))
    count += 1
    
durschschnitt = (value/count)


total = 0
count = 0
for x in df:
    if count > 0:
        total += pow((float(x[2])-durschschnitt),2)
    count += 1

total /= count-1

    
    
#print("df: ",df)
print("Anzahl: ",count)
print("Summe: ",value)
print("Total dif: ",dif)
print("Relativ dif: ",dif/count)
print("Durchschnitt: ",durschschnitt)
print("Standartabweichung: ", math.sqrt(total))
print("Rows Test: ",train.nrows)
print("Rows Prediction: ",pred.nrows)



Anzahl:  39390
Summe:  149791
Total dif:  32604.321447069557
Relativ dif:  0.8277309329035175
Durchschnitt:  3.802767199796903
Standartabweichung:  1.1048309297359762
Rows Test:  39389
Rows Prediction:  39389


In [ ]:
print(df[0:10])